In [41]:
from simpleai.search import CspProblem, backtrack, hill_climbing, simulated_annealing, genetic, MOST_CONSTRAINED_VARIABLE, LEAST_CONSTRAINING_VALUE
from simpleai.search.csp import min_conflicts
import time
import random

In [50]:
# Định nghĩa CSP cho bài toán 5 hậu
# Mỗi biến Q0..Q4 đại diện cho một hàng trên bàn cờ 5x5.
# Giá trị của biến là số cột (0..4) nơi đặt quân hậu trên hàng đó.
variables = [f"Q{i}" for i in range(5)]
domains = {v: list(range(5)) for v in variables}  # Mỗi Q có domain {0,1,2,3,4}

constraints = []
# Ràng buộc: không có 2 quân hậu nào tấn công nhau
for i in range(5):
    for j in range(i + 1, 5):
        v1, v2 = f"Q{i}", f"Q{j}"

        # closure để "ghi nhớ" i, j khi tạo constraint
        def make_constraint(i=i, j=j):
            def constraint(variables_tuple, values_tuple):
                col_i, col_j = values_tuple
                # Ràng buộc 1: Không cùng cột
                if col_i == col_j:
                    return False
                # Ràng buộc 2: Không cùng đường chéo
                # (chênh lệch hàng = chênh lệch cột => cùng đường chéo)
                return abs(i - j) != abs(col_i - col_j)
            return constraint

        # Gắn constraint này vào cặp biến (Q_i, Q_j)
        constraints.append(((v1, v2), make_constraint()))

# Tạo đối tượng CSP cho bài toán
problem = CspProblem(variables, domains, constraints)

# Giải bằng backtracking
solution = backtrack(problem)

# Hàm in bàn cờ
def print_board(positions):
    if positions is None:
        print("Không tìm được lời giải.")
        return
    n = 5
    # Tạo bảng n x n toàn '.' (ô trống)
    board = [['.' for _ in range(n)] for _ in range(n)]
    # Với mỗi biến 'Qk', ta tìm hàng = k-1 (chỉ số từ 0), cột = positions[var]-1
    # và đặt ký hiệu 'Q' vào ô đó.
    for var in variables:
        row = int(var[1:]) - 1 # 'Q1' -> row index 0
        col = positions[var] - 1 # cột lưu trong positions là 1..5 -> chuyển về 0-based
        board[row][col] = 'Q'
    for row in board: # In từng dòng của bàn cờ
        print(' '.join(row))

print("Lời giải tìm được:")
print(solution)
print("Bàn cờ:")
print_board(solution)


Lời giải tìm được:
{'Q0': 0, 'Q1': 2, 'Q2': 4, 'Q3': 1, 'Q4': 3}
Bàn cờ:
. Q . . .
. . . Q .
Q . . . .
. . Q . .
. . . . Q


In [49]:
# Định nghĩa CSP cho bài toán 5 hậu
# Mỗi biến Q0..Q4 đại diện cho một hàng trên bàn cờ 5x5.
# Giá trị của biến là số cột (0..4) nơi đặt quân hậu trên hàng đó.
variables = [f"Q{i}" for i in range(5)]
domains = {v: list(range(5)) for v in variables}  # Mỗi Q có domain {0,1,2,3,4}

constraints = []
# Ràng buộc: không có 2 quân hậu nào tấn công nhau
for i in range(5):
    for j in range(i + 1, 5):
        v1, v2 = f"Q{i}", f"Q{j}"

        # closure để "ghi nhớ" i, j khi tạo constraint
        def make_constraint(i=i, j=j):
            def constraint(variables_tuple, values_tuple):
                col_i, col_j = values_tuple
                # Ràng buộc 1: Không cùng cột
                if col_i == col_j:
                    return False
                # Ràng buộc 2: Không cùng đường chéo
                # (chênh lệch hàng = chênh lệch cột => cùng đường chéo)
                return abs(i - j) != abs(col_i - col_j)
            return constraint

        # Gắn constraint này vào cặp biến (Q_i, Q_j)
        constraints.append(((v1, v2), make_constraint()))

# Tạo đối tượng CSP cho bài toán
problem = CspProblem(variables, domains, constraints)

# Hàm in bàn cờ từ lời giải
def print_board(solution):
    if solution is None:
        print("Không tìm thấy lời giải.")
        return
    n = len(solution)
    # Khởi tạo bàn cờ n x n toàn dấu '.'
    board = [["." for _ in range(n)] for _ in range(n)]
    # Đặt quân hậu 'Q' theo lời giải
    for row in range(n):
        col = solution[f"Q{row}"]
        board[row][col] = "Q"
    # In ra bàn cờ
    for r in board:
        print(" ".join(r))

# Hàm thử nghiệm với từng chiến lược heuristic
def test_strategy(var_heuristic=None, val_heuristic=None, label="Mặc định"):
    start = time.time()
    sol = backtrack(problem,
                    variable_heuristic=var_heuristic,
                    value_heuristic=val_heuristic)
    elapsed = time.time() - start
    print(f"\nChiến lược: {label}")
    print("Lời giải:", sol)
    print_board(sol)
    print(f"Thời gian: {elapsed:.6f} giây")
    return elapsed

# Thử nghiệm các chiến lược khác nhau
# Không dùng heuristic (thuần backtracking)
time_default = test_strategy(label="Không heuristic")

# Heuristic chọn biến có ít giá trị hợp lệ nhất trước (MCV)
time_mcv = test_strategy(var_heuristic=MOST_CONSTRAINED_VARIABLE,
                             label="MOST_CONSTRAINED_VARIABLE")

# Heuristic chọn giá trị ít gây xung đột nhất trước (LCV)
time_lcv = test_strategy(val_heuristic=LEAST_CONSTRAINING_VALUE,
                             label="LEAST_CONSTRAINING_VALUE")

# In và So sánh kết quả giữa các heuristic
print("\n--- So sánh thời gian ---")
print(f"Không heuristic: {time_default:.6f} giây")
print(f"MCV: {time_mcv:.6f} giây")
print(f"LCV: {time_lcv:.6f} giây")

# Đánh giá chiến lược nào nhanh nhất
best_time = min(time_default, time_mcv, time_lcv)
if best_time == time_mcv:
    print("→ MOST_CONSTRAINED_VARIABLE tốt nhất.")
elif best_time == time_lcv:
    print("→ LEAST_CONSTRAINING_VALUE tốt nhất.")
else:
    print("→ Không heuristic lại nhanh nhất (trường hợp đặc biệt).")


Chiến lược: Không heuristic
Lời giải: {'Q0': 0, 'Q1': 2, 'Q2': 4, 'Q3': 1, 'Q4': 3}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .
Thời gian: 0.001064 giây

Chiến lược: MOST_CONSTRAINED_VARIABLE
Lời giải: {'Q0': 0, 'Q3': 1, 'Q1': 2, 'Q2': 4, 'Q4': 3}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .
Thời gian: 0.001036 giây

Chiến lược: LEAST_CONSTRAINING_VALUE
Lời giải: {'Q0': 0, 'Q1': 2, 'Q2': 4, 'Q3': 1, 'Q4': 3}
Q . . . .
. . Q . .
. . . . Q
. Q . . .
. . . Q .
Thời gian: 0.001153 giây

--- So sánh thời gian ---
Không heuristic: 0.001064 giây
MCV: 0.001036 giây
LCV: 0.001153 giây
→ MOST_CONSTRAINED_VARIABLE tốt nhất.


In [51]:
# Định nghĩa CSP cho bài toán 5 hậu
# Mỗi biến Q0..Q4 đại diện cho một hàng trên bàn cờ 5x5.
# Giá trị của biến là số cột (0..4) nơi đặt quân hậu trên hàng đó.
variables = [f"Q{i}" for i in range(5)]
domains = {v: list(range(5)) for v in variables}  # Mỗi Q có domain {0,1,2,3,4}

constraints = []
# Ràng buộc: không có 2 quân hậu nào tấn công nhau
for i in range(5):
    for j in range(i + 1, 5):
        v1, v2 = f"Q{i}", f"Q{j}"

        # closure để "ghi nhớ" i, j khi tạo constraint
        def make_constraint(i=i, j=j):
            def constraint(variables_tuple, values_tuple):
                col_i, col_j = values_tuple
                # Ràng buộc 1: Không cùng cột
                if col_i == col_j:
                    return False
                # Ràng buộc 2: Không cùng đường chéo
                # (chênh lệch hàng = chênh lệch cột => cùng đường chéo)
                return abs(i - j) != abs(col_i - col_j)
            return constraint

        # Gắn constraint này vào cặp biến (Q_i, Q_j)
        constraints.append(((v1, v2), make_constraint()))

# Tạo đối tượng CSP cho bài toán
problem = CspProblem(variables, domains, constraints)

# Hàm in bàn cờ từ solution
def print_board(solution):
    if solution is None:
        print("Không tìm được lời giải.")
        return
    n = 5
    board = [['.' for _ in range(n)] for _ in range(n)]
    for var in variables:
        row = int(var[1:]) - 1  # Hàng (0-4)
        col = solution[var] - 1  # Cột (0-4)
        board[row][col] = 'Q'
    for row in board:
        print(' '.join(row))

# Thử nghiệm với AC-3 và không AC-3
# (a) Có AC-3 (inference=True)
start_ac3 = time.time()
solution_ac3 = backtrack(problem, inference=True)
time_ac3 = time.time() - start_ac3

print("- Lời giải với AC-3 (inference=True):")
print(solution_ac3)
print_board(solution_ac3)
print(f"Thời gian: {time_ac3:.6f} giây\n")

# (b) Không AC-3 (inference=False)
start_no_ac3 = time.time()
solution_no_ac3 = backtrack(problem, inference=False)
time_no_ac3 = time.time() - start_no_ac3

print("- Lời giải không AC-3 (inference=False):")
print(solution_no_ac3)
print_board(solution_no_ac3)
print(f"Thời gian: {time_no_ac3:.6f} giây\n")

# In và So sánh kết quả
if time_ac3 < time_no_ac3:
    print("Sử dụng AC-3 tốt hơn (nhanh hơn).")
elif time_no_ac3 < time_ac3:
    print("Không sử dụng AC-3 tốt hơn (nhanh hơn).")
else:
    print("Cả hai chiến lược có thời gian như nhau.")

- Lời giải với AC-3 (inference=True):
{'Q0': 0, 'Q1': 2, 'Q2': 4, 'Q3': 1, 'Q4': 3}
. Q . . .
. . . Q .
Q . . . .
. . Q . .
. . . . Q
Thời gian: 0.001237 giây

- Lời giải không AC-3 (inference=False):
{'Q0': 0, 'Q1': 2, 'Q2': 4, 'Q3': 1, 'Q4': 3}
. Q . . .
. . . Q .
Q . . . .
. . Q . .
. . . . Q
Thời gian: 0.000567 giây

Không sử dụng AC-3 tốt hơn (nhanh hơn).


In [52]:
# Định nghĩa lớp bài toán 5 hậu cho local search
class NQueensProblem(SearchProblem):
    def __init__(self, n=5):
        self.n = n
        # Trạng thái ban đầu ngẫu nhiên: mỗi hàng đặt hậu ở một cột
        self.initial_state = tuple(random.randint(1, n) for _ in range(n))

    def actions(self, state):
        """Danh sách hành động: di chuyển hậu ở một hàng sang cột khác"""
        actions = []
        for row in range(len(state)):
            for col in range(1, self.n + 1):
                if col != state[row]:  # chỉ di chuyển sang cột khác
                    actions.append((row, col))
        return actions

    def result(self, state, action):
        """Thực hiện hành động: đặt hậu ở hàng row sang cột col"""
        row, col = action
        new_state = list(state)
        new_state[row] = col
        return tuple(new_state)

    def value(self, state):
        """Hàm giá trị: số xung đột càng ít càng tốt → dùng -conflicts"""
        conflicts = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                    conflicts += 1
        return -conflicts  # simpleai tối đa hóa, nên dùng giá trị âm

    # Các hàm cho Genetic Algorithm
    def generate_random_state(self):
        return tuple(random.randint(1, self.n) for _ in range(self.n))

    def crossover(self, state1, state2):
        point = random.randint(1, self.n - 1)
        return state1[:point] + state2[point:]

    def mutate(self, state):
        row = random.randint(0, self.n - 1)
        col = random.randint(1, self.n)
        new_state = list(state)
        new_state[row] = col
        return tuple(new_state)

# Hàm in bàn cờ
def print_board(state):
    if state is None:
        print("Không tìm được lời giải.")
        return
    n = len(state)
    board = [['.' for _ in range(n)] for _ in range(n)]
    for row, col in enumerate(state):
        board[row][col - 1] = 'Q'
    for row in board:
        print(' '.join(row))

# Chạy thử nghiệm với các thuật toán
n = 5
problem = NQueensProblem(n)

# Hill Climbing
start_hc = time.time()
solution_hc = hill_climbing(problem)
time_hc = time.time() - start_hc
state_hc = solution_hc.state if solution_hc else None

print("-- Hill Climbing --")
print("Lời giải:", state_hc)
print_board(state_hc)
print(f"Thời gian: {time_hc:.6f} giây\n")

# Simulated Annealing
start_sa = time.time()
solution_sa = simulated_annealing(problem, schedule=_exp_schedule)
time_sa = time.time() - start_sa
state_sa = solution_sa.state if solution_sa else None

print("-- Simulated Annealing --")
print("Lời giải:", state_sa)
print_board(state_sa)
print(f"Thời gian: {time_sa:.6f} giây\n")

# Genetic Algorithm
start_ga = time.time()
solution_ga = genetic(problem, population_size=50, mutation_chance=0.2)
time_ga = time.time() - start_ga
state_ga = solution_ga.state if solution_ga else None

print("-- Genetic Algorithm --")
print("Lời giải:", state_ga)
print_board(state_ga)
print(f"Thời gian: {time_ga:.6f} giây\n")


-- Hill Climbing --
Lời giải: (5, 2, 4, 1, 3)
. . . . Q
. Q . . .
. . . Q .
Q . . . .
. . Q . .
Thời gian: 0.000344 giây

-- Simulated Annealing --
Lời giải: (5, 2, 4, 1, 5)
. . . . Q
. Q . . .
. . . Q .
Q . . . .
. . . . Q
Thời gian: 0.000184 giây

-- Genetic Algorithm --
Lời giải: (4, 2, 4, 5, 1)
. . . Q .
. Q . . .
. . . Q .
. . . . Q
Q . . . .
Thời gian: 0.001141 giây

